In [25]:
import numpy as np
import plotly.graph_objs as go
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from rdkit import Chem
import os
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from matchms.importing import load_from_msp
from matchms.logging_functions import set_matchms_logger_level
from utils import is_spectrum_for_compound

set_matchms_logger_level('ERROR')


In [26]:
matchms_scores = pd.read_csv("../data/output_matching/matchms/matchms_tol_0.01_1%I_all_peaks_with_0s.tsv", sep="\t")

matchms_scores = matchms_scores[matchms_scores.apply(lambda x: is_spectrum_for_compound(x['query'], x['reference']), axis=1)]

molecules = Chem.SDMolSupplier("../../data/RECETOX_GC-EI-MS_20201028.sdf")
predicted_spectra = list(load_from_msp("../data/filtered/simulated_matchms_filter_1%I_all_peaks.msp"))
spectra_metadata= pd.DataFrame.from_dict([x.metadata for x in predicted_spectra])
spectra_metadata.rename(columns={'compound_name': 'query'}, inplace=True)

In [27]:
merged = matchms_scores.merge(spectra_metadata, on="query", how="inner")

In [28]:
# Create a scatter plot
fig = go.Figure()
fig.add_trace(go.Scatter(
    y=matchms_scores['CosineHungarian_0.01_0.0_1.0_matches'],
    x=matchms_scores['CosineHungarian_0.01_0.0_1.0_scores'],
    mode='markers',
    marker=dict(
        size=5,
        color='blue',
        opacity=0.5
    ),
    name='Matched ions'
))
fig.update_layout(
    title='Scatter plot of matched ions and scores',
    xaxis_title='score',
    yaxis_title='matches'
)
fig.show()
# node size should be the experimental spectra, color the fraction of predicted spectra